# Recurrent Neural Network

In [1]:
# imports 
import torch 
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# HyperParams
seq_length    = 28
input_size    = 28
hidden_size   = 128
num_layers    = 2
num_classes   = 10
batch_size    = 100
num_epochs    = 2 
learning_rate = 0.01

In [4]:
# MNIST 
train_dataset = datasets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = datasets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [29]:
# nn.LSTM?

In [13]:
# rnn = nn.LSTM(10, 20, 2)
# input = Variable(torch.randn(5, 3, 10))
# # print input.size(0)
# h0 = Variable(torch.randn(2, 3, 20))
# c0 = Variable(torch.randn(2, 3, 20))
# output, hn = rnn(input, (h0, c0))
# print output

In [16]:
# RNN 
class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,num_classes):
        super(RNN,self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
    
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size,num_classes)
        
    def forward(self,x):
        h0 = Variable(torch.zeros(self.num_layers,x.size(0),self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers,x.size(0),self.hidden_size))
        
        out,_ = self.lstm(x,(h0,c0))
        
        #hidden state of last time_step
        out = self.fc(out[:,-1,:])
        return out

In [17]:
# RNN
rnn = RNN(input_size,hidden_size,num_layers,num_classes)

In [18]:
# Loss & Optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(),lr=learning_rate)

In [22]:
# Train
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images = Variable(images.view(-1,seq_length,input_size))
        labels = Variable(labels)
        
        #Forward->loss->backprop->optimize
        optimizer.zero_grad()
        outputs = rnn(images)
        
        loss = criterion(outputs,labels)
        
        loss.backward()
        
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/2], Step [10/600], Loss: 0.1480
Epoch [1/2], Step [20/600], Loss: 0.1906
Epoch [1/2], Step [30/600], Loss: 0.0748
Epoch [1/2], Step [40/600], Loss: 0.1605
Epoch [1/2], Step [50/600], Loss: 0.3945
Epoch [1/2], Step [60/600], Loss: 0.2286
Epoch [1/2], Step [70/600], Loss: 0.1657
Epoch [1/2], Step [80/600], Loss: 0.1875
Epoch [1/2], Step [90/600], Loss: 0.1135
Epoch [1/2], Step [100/600], Loss: 0.0716
Epoch [1/2], Step [110/600], Loss: 0.1758
Epoch [1/2], Step [120/600], Loss: 0.2707
Epoch [1/2], Step [130/600], Loss: 0.1936
Epoch [1/2], Step [140/600], Loss: 0.0796
Epoch [1/2], Step [150/600], Loss: 0.0674
Epoch [1/2], Step [160/600], Loss: 0.1413
Epoch [1/2], Step [170/600], Loss: 0.1469
Epoch [1/2], Step [180/600], Loss: 0.1939
Epoch [1/2], Step [190/600], Loss: 0.0997
Epoch [1/2], Step [200/600], Loss: 0.1290
Epoch [1/2], Step [210/600], Loss: 0.2057
Epoch [1/2], Step [220/600], Loss: 0.1666
Epoch [1/2], Step [230/600], Loss: 0.0410
Epoch [1/2], Step [240/600], Loss: 0.0919
E

In [25]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, seq_length, input_size))
    outputs = rnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total)) 


Test Accuracy of the model on the 10000 test images: 97 %


In [23]:

# Save the Model
# torch.save(rnn.state_dict(), 'rnn.pkl')